In [1]:
import urllib
from IPython.display import Markdown as md

### change to reflect your notebook
_nb_loc = "09_deploying/09d_bytes.ipynb"
_nb_title = "이미지 바이트 다루기"
_nb_message = ""

### no need to change any of this
_icons=["https://raw.githubusercontent.com/ychoi-kr/practical-ml-vision-book-ko/master/logo-cloud.png", "https://www.tensorflow.org/images/colab_logo_32px.png", "https://www.tensorflow.org/images/GitHub-Mark-32px.png", "https://www.tensorflow.org/images/download_logo_32px.png"]
_links=["https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?" + urllib.parse.urlencode({"name": _nb_title, "download_url": "https://github.com/ychoi-kr/practical-ml-vision-book-ko/raw/master/"+_nb_loc}), "https://colab.research.google.com/github/ychoi-kr/practical-ml-vision-book-ko/blob/master/{0}".format(_nb_loc), "https://github.com/ychoi-kr/practical-ml-vision-book-ko/blob/master/{0}".format(_nb_loc), "https://raw.githubusercontent.com/ychoi-kr/practical-ml-vision-book-ko/master/{0}".format(_nb_loc)]
md("""<table class="tfo-notebook-buttons" align="left"><td><a target="_blank" href="{0}"><img src="{4}"/>AI Platform Notebook에서 실행</a></td><td><a target="_blank" href="{1}"><img src="{5}" />Google Colab에서 실행</a></td><td><a target="_blank" href="{2}"><img src="{6}" />GitHub에서 소스 보기</a></td><td><a href="{3}"><img src="{7}" />노트북 내려받기</a></td></table><br/><br/><h1>{8}</h1>{9}""".format(_links[0], _links[1], _links[2], _links[3], _icons[0], _icons[1], _icons[2], _icons[3], _nb_title, _nb_message))

<table class="tfo-notebook-buttons" align="left"><td><a target="_blank" href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?name=%EC%9D%B4%EB%AF%B8%EC%A7%80+%EB%B0%94%EC%9D%B4%ED%8A%B8+%EB%8B%A4%EB%A3%A8%EA%B8%B0&download_url=https%3A%2F%2Fgithub.com%2Fychoi-kr%2Fpractical-ml-vision-book-ko%2Fraw%2Fmaster%2F09_deploying%2F09d_bytes.ipynb"><img src="https://raw.githubusercontent.com/ychoi-kr/practical-ml-vision-book-ko/master/logo-cloud.png"/>AI Platform Notebook에서 실행</a></td><td><a target="_blank" href="https://colab.research.google.com/github/ychoi-kr/practical-ml-vision-book-ko/blob/master/09_deploying/09d_bytes.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Google Colab에서 실행</a></td><td><a target="_blank" href="https://github.com/ychoi-kr/practical-ml-vision-book-ko/blob/master/09_deploying/09d_bytes.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />GitHub에서 소스 보기</a></td><td><a href="https://raw.githubusercontent.com/ychoi-kr/practical-ml-vision-book-ko/master/09_deploying/09d_bytes.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />노트북 내려받기</a></td></table><br/><br/><h1>이미지 바이트 다루기</h1>

# 이미지 바이트 다루기

이 노트북에서는 앞서 훈련 및 저장한 모델(7장)을 가지고 시작한다.
편의상 이 모델을 공개 버킷 [gs://practical-ml-vision-book/flowers_5_trained](gs://practical-ml-vision-book/flowers_5_trained)에 두었다.

우리가 원하는 것은 네트워크를 통해 바이트를 직접 처리하는 것이다. 이렇게 하면 클라이언트가 Google Cloud Storage에 이미지를 저장할 필요가 없다.

## GPU 활성화 및 도우미 함수 셋업

이 저장소의 노트북들은 GPU를 사용하면 더 빨리 실행된다.

코랩에서:

- 수정 → 노트 설정 메뉴로 이동
- 하드웨어 가속기 드롭다운에서 GPU를 선택

Cloud AI Platform Notebooks에서:

- [https://console.cloud.google.com/ai-platform/notebooks](https://console.cloud.google.com/ai-platform/notebooks)로 이동
- GPU를 포함한 인스턴스를 생성하거나, 인스턴스를 선택하고 GPU를 추가

다음으로, 텐서플로로 GPU에 연결할 수 있는지 확인.

In [ ]:
import tensorflow as tf
print('TensorFlow version' + tf.version.VERSION)
print('Built with GPU support? ' + ('Yes!' if tf.test.is_built_with_cuda() else 'Noooo!'))
print('There are {} GPUs'.format(len(tf.config.experimental.list_physical_devices("GPU"))))
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

## 체크포인트에서 읽기

시그니처만이 아닌 전체 모델을 원하므로, 저장된 모델이 아닌 *체크포인트*에서 시작한다.

In [ ]:
import os
import shutil
import tensorflow as tf

CHECK_POINT_DIR='gs://practical-ml-vision-book/flowers_5_trained/chkpts'
model = tf.keras.models.load_model(CHECK_POINT_DIR)
print(model.summary())

Model: "flower_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random/center_crop (RandomCr (None, 224, 224, 3)       0         
_________________________________________________________________
random_lr_flip/none (RandomF (None, 224, 224, 3)       0         
_________________________________________________________________
mobilenet_embedding (KerasLa (None, 1280)              2257984   
_________________________________________________________________
dense_hidden (Dense)         (None, 32)                40992     
_________________________________________________________________
flower_prob (Dense)          (None, 5)                 165       
Total params: 2,299,141
Trainable params: 2,265,029
Non-trainable params: 34,112
_________________________________________________________________
None


In [ ]:
IMG_HEIGHT = 345
IMG_WIDTH = 345
IMG_CHANNELS = 3
CLASS_NAMES = 'daisy dandelion roses sunflowers tulips'.split()

def read_from_jpegfile(filename):
    img_bytes = tf.io.read_file(filename)
    return img_bytes
    
def preprocess(img_bytes):
    img = tf.image.decode_jpeg(img_bytes, channels=IMG_CHANNELS)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return tf.image.resize_with_pad(img, IMG_HEIGHT, IMG_WIDTH)

In [ ]:
filenames = [
    'gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dandelion/9818247_e2eac18894.jpg',
    'gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg',
    'gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg',
    'gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/tulips/8733586143_3139db6e9e_n.jpg',
    'gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/tulips/8713397358_0505cc0176_n.jpg'
]
for filename in filenames:
    img_bytes = read_from_jpegfile(filename)
    img = preprocess(img_bytes)
    img = tf.expand_dims(img, axis=0)
    pred = model.predict(img)
    print(pred)

[[0.3507376  0.3983379  0.02309519 0.07595135 0.15187794]]
[[3.1782882e-05 9.9996090e-01 5.1874702e-07 3.2268999e-06 3.5444552e-06]]
[[9.9471879e-01 3.5855272e-03 2.1374140e-05 1.5876008e-03 8.6639280e-05]]
[[1.5454909e-03 2.2907292e-04 3.6099207e-02 3.1195192e-03 9.5900667e-01]]
[[4.7941930e-06 3.9310632e-07 5.8220904e-02 9.1497981e-07 9.4177294e-01]]


## 클라이언트의 바이트를 처리할 시그니처 내보내기

In [ ]:
@tf.function(input_signature=[tf.TensorSpec([None,], dtype=tf.string)])
def predict_bytes(img_bytes):
    input_images = tf.map_fn(
        preprocess,
        img_bytes,
        fn_output_signature=tf.float32
    )
    batch_pred = model(input_images) # same as model.predict()
    top_prob = tf.math.reduce_max(batch_pred, axis=[1])
    pred_label_index = tf.math.argmax(batch_pred, axis=1)
    pred_label = tf.gather(tf.convert_to_tensor(CLASS_NAMES), pred_label_index)
    return {
        'probability': top_prob,
        'flower_type_int': pred_label_index,
        'flower_type_str': pred_label
    }

@tf.function(input_signature=[tf.TensorSpec([None,], dtype=tf.string)])
def predict_filename(filenames):
    img_bytes = tf.map_fn(
        tf.io.read_file,
        filenames
    )
    result = predict_bytes(img_bytes)
    result['filename'] = filenames
    return result

shutil.rmtree('export', ignore_errors=True)
os.mkdir('export')
model.save('export/flowers_model3',
          signatures={
              'serving_default': predict_filename,
              'from_bytes': predict_bytes
          })

INFO:tensorflow:Assets written to: export/flowers_model3/assets


INFO:tensorflow:Assets written to: export/flowers_model3/assets


In [ ]:
!saved_model_cli show --tag_set serve --dir export/flowers_model3

The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
SignatureDef key: "__saved_model_init_op"
SignatureDef key: "from_bytes"
SignatureDef key: "serving_default"


In [ ]:
!saved_model_cli show --tag_set serve --dir export/flowers_model3 --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['filenames'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: serving_default_filenames:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['filename'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: StatefulPartitionedCall_1:0
  outputs['flower_type_int'] tensor_info:
      dtype: DT_INT64
      shape: (-1)
      name: StatefulPartitionedCall_1:1
  outputs['flower_type_str'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: StatefulPartitionedCall_1:2
  outputs['probability'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1)
      name: StatefulPartitionedCall_1:3
Method name is: tensorflow/serving/predict


In [ ]:
!saved_model_cli show --tag_set serve --dir export/flowers_model3 --signature_def from_bytes

The given SavedModel SignatureDef contains the following input(s):
  inputs['img_bytes'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: from_bytes_img_bytes:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['flower_type_int'] tensor_info:
      dtype: DT_INT64
      shape: (-1)
      name: StatefulPartitionedCall:0
  outputs['flower_type_str'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: StatefulPartitionedCall:1
  outputs['probability'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1)
      name: StatefulPartitionedCall:2
Method name is: tensorflow/serving/predict


## 이미지 바이트 전송

GCS에 중간 파일이 필요하지 않다. 우리는 단순히 파이썬의 파일 읽기 메서드를 사용한다.

In [ ]:
!gsutil cp gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg /tmp/test.jpg

Copying gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg...
/ [1 files][ 19.4 KiB/ 19.4 KiB]                                                
Operation completed over 1 objects/19.4 KiB.                                     


In [ ]:
with open('/tmp/test.jpg', 'rb') as ifp:
    img_bytes = ifp.read()
    serving_fn = tf.keras.models.load_model('./export/flowers_model3').signatures['from_bytes']
    pred = serving_fn(tf.convert_to_tensor([img_bytes]))
    print(pred)

{'probability': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.9947188], dtype=float32)>, 'flower_type_str': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'daisy'], dtype=object)>, 'flower_type_int': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([0])>}


## CAIP에 바이트 처리 모델 배포

In [ ]:
%%bash
BUCKET="ai-analytics-solutions-mlvisionbook"  # CHANGE
gsutil -m cp -r ./export/flowers_model3 gs://${BUCKET}/flowers_model3

Copying file://./export/flowers_model3/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
Copying file://./export/flowers_model3/variables/variables.index [Content-Type=application/octet-stream]...
Copying file://./export/flowers_model3/saved_model.pb [Content-Type=application/octet-stream]...
/ [3/3 files][ 10.7 MiB/ 10.7 MiB] 100% Done                                    
Operation completed over 3 objects/10.7 MiB.                                     


In [ ]:
%%bash
BUCKET="ai-analytics-solutions-mlvisionbook" # CHANGE
./vertex_deploy.sh \
--endpoint_name=bytes \
--model_name=bytes \
--model_location=gs://${BUCKET}/flowers_model3

Deploying model bytes
Creating bytes endpoint now.
The endpoint_id is 7318683646011899904
Uploading bytes model now.
The model_id is 2990680423643742208
Deploying model now


Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [1561614649575604224]...
.....done.
Created Vertex AI endpoint: projects/563535018348/locations/us-central1/endpoints/7318683646011899904.
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [8091834109262823424]...
.....done.
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [3867457658789298176]...
..............................................................................................................................................................................................................................................................................................

## 중요: 이 셀을 변경

위의 엔드포인트 ID와 배포된 모델 ID를 기록해 둔다. 아래 셀에 입력한다.

In [ ]:
# 배포한 모델에 맞게 아래를 수정할 것
import os
os.environ['ENDPOINT_ID'] = '7318683646011899904' # 바
os.environ['MODEL_ID'] = '6992243041771716608' # 꿔
os.environ['PROJECT'] = 'ai-analytics-solutions' # 주
os.environ['BUCKET'] = 'ai-analytics-solutions-mlvisionbook' # 세
os.environ['REGION'] = 'us-central1' # 요

In [ ]:
%%bash
gsutil cp gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg /tmp/test1.jpg
gsutil cp gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/tulips/8713397358_0505cc0176_n.jpg /tmp/test2.jpg

Copying gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg...
/ [1 files][ 19.4 KiB/ 19.4 KiB]                                                
Operation completed over 1 objects/19.4 KiB.                                     
Copying gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/tulips/8713397358_0505cc0176_n.jpg...
/ [1 files][ 34.6 KiB/ 34.6 KiB]                                                
Operation completed over 1 objects/34.6 KiB.                                     


base-64로 인코딩된 데이터를 전달하는 방법에 유의하라.

In [ ]:
# 파이썬에서 호출
import base64
import json
from oauth2client.client import GoogleCredentials
import requests

PROJECT = "ai-analytics-solutions"  # 변경
REGION = "us-central1"  # 이 리전에 GPU/TPU 쿼타가 있는지 확인할 것
ENDPOINT_ID = "7318683646011899904"

def b64encode(filename):
    with open(filename, 'rb') as ifp:
        img_bytes = ifp.read()
        return base64.b64encode(img_bytes)

token = GoogleCredentials.get_application_default().get_access_token().access_token
api = "https://{}-aiplatform.googleapis.com/v1/projects/{}/locations/{}/endpoints/{}:predict".format(
    REGION, PROJECT, REGION, ENDPOINT_ID)
headers = {"Authorization": "Bearer " + token }
data = {
    "signature_name": "from_bytes",  # currently bugged
    "instances": [
        {
            "img_bytes": {"b64": b64encode('/tmp/test1.jpg')}
        },
        {
            "img_bytes": {"b64": b64encode('/tmp/test2.jpg')}
        },
    ]
}
response = requests.post(api, json=data, headers=headers)
print(response.content)

b'{\n  "error": {\n    "code": 400,\n    "message": "Invalid JSON payload received. Unknown name \\"signature_name\\": Cannot find field.",\n    "status": "INVALID_ARGUMENT",\n    "details": [\n      {\n        "@type": "type.googleapis.com/google.rpc.BadRequest",\n        "fieldViolations": [\n          {\n            "description": "Invalid JSON payload received. Unknown name \\"signature_name\\": Cannot find field."\n          }\n        ]\n      }\n    ]\n  }\n}\n'


## 라이선스
Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.